In [147]:
import sqlite3
import pandas as pd
import plotly.express as px
import sys
import plotly.graph_objects as go

sys.path.append('../')
from functions_env import DB_PATH


conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

import io

import folium
from PIL import Image, ImageDraw, ImageFont

In [148]:
df_deathcity = pd.read_csv('/Users/charlesdedampierre/Desktop/cultura_project/raw_data/wikidata_data/deathcity_location.csv')
df_deathcity = df_deathcity[['deathcity_wiki_id', 'location']]
df_deathcity = df_deathcity.dropna()
df_deathcity  = df_deathcity[df_deathcity['location'].str.contains('Point')]
df_deathcity['city_longitude'] = df_deathcity['location'].apply(lambda x: x.split(' ')[0].split('(')[1]).astype(float)
df_deathcity['city_latitude'] = df_deathcity['location'].apply(lambda x: x.split(' ')[1].split(')')[0]).astype(float)
df_deathcity = df_deathcity.drop('location', axis=1)
df_deathcity = df_deathcity.rename(columns={'deathcity_wiki_id':'deathcity_wikidata_id'})

df_ind_deathcity = pd.read_sql_query("SELECT * FROM individual_deathcity", conn)
df_ind_regions = pd.read_sql_query("SELECT * FROM individuals_regions", conn)
df_fin_deathcity = pd.merge(df_deathcity, df_ind_deathcity, on = 'deathcity_wikidata_id')


df_year = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_year['productive_year'] = df_year['birthyear'] + 35
df_year = df_year[['individual_wikidata_id', 'productive_year']]
df_year = df_year.dropna()
df_year['decade'] = df_year['productive_year'].apply(lambda x: round(x / 10) * 10)

df_region = pd.merge(df_ind_regions, df_year)

In [159]:
def make_picture(df_region, 
                 re_region = 'Chinese world',
                 min_date= 1400,
                 max_date = 1850, 
                 min_opacity = 0.9,
                 blur = 11, 
                 radius = 10, 
                 text_position = (320, 40), 
                 left_padding =  0.15, 
                 right_padding = 0.3,
                 zoom_start = 4.4,
                 image_text = None):

    df_map =df_region[df_region['region_name']==re_region]
    df_map = df_map[(df_map['decade']>=min_date)&(df_map['decade']<=max_date)]

    df_map = pd.merge(df_map, df_fin_deathcity, on = 'individual_wikidata_id')
    data_loc = df_map.groupby(['city_latitude', 'city_longitude'])['individual_wikidata_id'].count().reset_index()
    m = folium.Map(location=data_loc[['city_latitude', 'city_longitude']].mean().to_list(), zoom_start=zoom_start)
    heat_data = [[row['city_latitude'], row['city_longitude']] for index, row in data_loc.iterrows()]
    HeatMap(heat_data, min_opacity=min_opacity, blur=blur, radius=radius).add_to(m)

    img_data = m._to_png(2)
    img = Image.open(io.BytesIO(img_data))

    # Crop on the right side
    right = int(img.width * right_padding)  # Adjust this value to control the amount to crop on the right side
    left = int(img.width * left_padding)  # Adjust this value to control the amount to crop on the left side
    top = 0
    width = img.width - right - left
    height = img.height

    cropped_img = img.crop((left, top, left + width, top + height))
    draw = ImageDraw.Draw(cropped_img)
    font = ImageFont.truetype("FreeMono.ttf", 70)  # You can specify the font file and size


    if image_text is None:
        # Define the text and position
        text = str(min_date) +'-'+str(max_date)
    else:
        text = image_text

    position = text_position  # X and Y coordinates where you want to place the text
    # Define the text color
    text_color = (0, 0, 0)  # RGB color (white in this example)

    # Add the text to the image
    draw.text(position, text, fill=text_color, font=font)
    cropped_img.save(f"figures/new_maps/{re_region}_{text}.png")

## Chinese World

In [161]:
make_picture(df_region, 
                 re_region = 'Chinese world',
                 min_date= -500,
                 max_date = 960, 
                 min_opacity = 0.9,
                 blur = 11, 
                 radius = 10, 
                 text_position = (320, 40), 
                 left_padding =  0.15, 
                 right_padding = 0.3,
                 image_text = '500BC-960',
                 zoom_start=4.4
                 )

In [162]:
make_picture(df_region, 
                 re_region = 'Chinese world',
                 min_date= 960,
                 max_date = 1368, 
                 min_opacity = 0.9,
                 blur = 11, 
                 radius = 10, 
                 text_position = (320, 40), 
                 left_padding =  0.15, 
                 right_padding = 0.3,
                zoom_start = 4.4)

In [192]:
make_picture(df_region, 
                 re_region = 'Chinese world',
                 min_date= 1368,
                 max_date = 1850, 
                 min_opacity = 0.9,
                 blur = 11, 
                 radius = 10, 
                 text_position = (320, 40), 
                 left_padding =  0.15, 
                 right_padding = 0.3,
                  zoom_start = 4.4)

## Western Europe

In [202]:
make_picture(df_region, 
                 re_region = 'Western Europe',
                 min_date= -800,
                 max_date = 1200, 
                 min_opacity = 0.5,
                 blur = 4, 
                 radius = 5, 
                 text_position = (320, 40), 
                 left_padding =  0.3, 
                 right_padding = 0.15,
                 image_text='800BC-1200',
                zoom_start = 4.2)

In [198]:
make_picture(df_region, 
                 re_region = 'Western Europe',
                 min_date= 1200,
                 max_date = 1850, 
                 min_opacity = 1,
                 blur = 4, 
                 radius = 2, 
                 text_position = (320, 40), 
                 left_padding =  0.3, 
                 right_padding = 0.15,
                zoom_start = 4)

## Japan

In [216]:
make_picture(df_region, 
                 re_region = 'Japan',
                 min_date= 0,
                 max_date = 1185, 
                 min_opacity = 0.5,
                 blur = 8, 
                 radius = 12, 
                 text_position = (320, 40), 
                 left_padding =  0.3, 
                 right_padding = 0.15,
                zoom_start = 5.2)

In [217]:
make_picture(df_region, 
                 re_region = 'Japan',
                 min_date= 1185,
                 max_date = 1850, 
                min_opacity = 0.8,
                 blur = 4, 
                 radius = 5,
                 text_position = (320, 40), 
                 left_padding =  0.3, 
                 right_padding = 0.15,
                zoom_start = 5.2)

In [153]:
"""from ipyleaflet import Map, Heatmap
re_region = 'Chinese world'
min_date= 1400
max_date = 1850
df_region = pd.merge(df_ind_regions, df_year)

df_map =df_region[df_region['region_name']==re_region]
df_map = df_map[(df_map['decade']>=min_date)&(df_map['decade']<=max_date)]

df_map = pd.merge(df_map, df_fin_deathcity, on = 'individual_wikidata_id')
data_loc = df_map.groupby(['city_latitude', 'city_longitude'])['individual_wikidata_id'].count().reset_index()

latitudes = data_loc['city_latitude'].values
longitudes = data_loc['city_longitude'].values
count = data_loc['individual_wikidata_id'].values

min_opacity = 0.7
blur = 12
radius = 10
zoom = 4

m = Map(center= data_loc[['city_latitude', 'city_longitude']].mean().to_list(), zoom=zoom)
#m = Map(zoom=3)

heatmap = Heatmap(locations=[[latitudes[i], longitudes[i], count[i]] for i in range(len(data_loc))],
                  radius=radius, min_opacity=min_opacity, blur=blur)

m.add_layer(heatmap)
m"""

"from ipyleaflet import Map, Heatmap\nre_region = 'Chinese world'\nmin_date= 1400\nmax_date = 1850\ndf_region = pd.merge(df_ind_regions, df_year)\n\ndf_map =df_region[df_region['region_name']==re_region]\ndf_map = df_map[(df_map['decade']>=min_date)&(df_map['decade']<=max_date)]\n\ndf_map = pd.merge(df_map, df_fin_deathcity, on = 'individual_wikidata_id')\ndata_loc = df_map.groupby(['city_latitude', 'city_longitude'])['individual_wikidata_id'].count().reset_index()\n\nlatitudes = data_loc['city_latitude'].values\nlongitudes = data_loc['city_longitude'].values\ncount = data_loc['individual_wikidata_id'].values\n\nmin_opacity = 0.7\nblur = 12\nradius = 10\nzoom = 4\n\nm = Map(center= data_loc[['city_latitude', 'city_longitude']].mean().to_list(), zoom=zoom)\n#m = Map(zoom=3)\n\nheatmap = Heatmap(locations=[[latitudes[i], longitudes[i], count[i]] for i in range(len(data_loc))],\n                  radius=radius, min_opacity=min_opacity, blur=blur)\n\nm.add_layer(heatmap)\nm"